# Testing The API

## Imports

In [78]:
from typing import List
from pydantic import BaseModel
from enum import Enum
from fastapi import FastAPI
from neo4j import GraphDatabase
from langchain.vectorstores import Neo4jVector
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import Neo4jVector
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA  # Q&A retrieval system.
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from operator import itemgetter
import uvicorn
import os
import langchain
from datetime import datetime
from pydantic import BaseModel


## Data Model

In [2]:
## DATA MODEL =============================================
class FeedbackEnum(str, Enum):
    """ 
        A Pydantic model representing feedback. 
    """
    like = 1
    neutral = 0
    dislike = -1

class Prompt(BaseModel):
    """
    A Pydantic model representing a LLM prompt
    """
    prompt: str

class Question(BaseModel):
    """
    A Pydantic model representing a search question. 
    """
    search_string: str

class ChunkMetadata(BaseModel):
    """
    A Pydantic model representing metadata.
    """
    chunk_size: int
    embedding_model: str
    chunk_order: int
    chunk_overlap: int
    chunk_id: int

class Document(BaseModel):
    """
    A Pydantic model representing a document.
    Wrapper around langchains langchain.schema.document.Document.
    """
    page_content: str
    metadata: ChunkMetadata

class Answer(BaseModel):
    """
    A Pydantic model representing an answer to a question.
    """
    context: List[Document]
    llm_prompt: str
    llm_answer: str
    language: str
    score: FeedbackEnum


## Constants

In [41]:
BASE_PROMPT_TEMPLATE = """
ANSWER THE QUESTION BASED ONLY ON THE FOLLOWING CONTEXT:
{context}

QUESTION: 
{question}

ANSWER IN THE FOLLOWING LANGUAGE: 
{language}

CLEARLY STATE IF THE ANSWER CANNOT BE FOUND IN THE CONTEXT ABOVE.
IF THE ANSWER CAN BE FOUND, REFERENCE THE CONTEXT. 
"""
# Define the configuration
RETRIEVER_SEARCH_CONFIG = {
    # "similarity" (default), "mmr", or "similarity_score_threshold".
    'search_type': 'similarity', 
    'search_kwargs': {
        # Amount of documents to return (default: 4).
        'k': 5, 
        # Amount of documents to pass to the MMR algorithm 
        # # (default: 20).
        'fetch_k': 50, 
        # Minimum relevance threshold for similarity_score_threshold.
        'score_threshold': 0, 
        # Diversity of results returned by MMR; 
        # # 1 for minimum diversity and 0 for maximum (default: 0.5).
        'lambda_mult': 0.25, 
        # Filter by document metadata.
        'filter': {'chunk_size': 500}
    }
}


## Global Variables

In [50]:
#### GLOBAL VARIABLES =================
langchain.verbose = False
langchain.debug = False
load_dotenv() 

# Instructor Embeddings
embeddings = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-xl", 
    cache_folder='./models'
)

# Graph from existing graph
neo4j_graph = Neo4jVector.from_existing_index(
    embedding=embeddings,
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USER"),
    password=os.getenv("NEO4J_PASSWORD"),
    index_name="vi_chunk_embedding_cosine",
    keyword_index_name="fts_Chunk_text",
    search_type="hybrid",
)

# Neo4j driver
driver = GraphDatabase.driver(
    os.getenv("NEO4J_URI"), 
    auth=(
        os.getenv("NEO4J_USER"), 
        os.getenv("NEO4J_PASSWORD")
    )
)

# Langchain retriever
neo4j_retriever = neo4j_graph.as_retriever(**RETRIEVER_SEARCH_CONFIG)

load INSTRUCTOR_Transformer
max_seq_length  512


In [6]:
## RETRIEVE DOCUMENTS =================
# Get docs from Neo4j
docs: List[langchain.schema.document.Document] = (
    neo4j_retriever.get_relevant_documents(
        question
    )
)

In [7]:
# Get Retriever Context 
context: List[Document] = [
    Document(
        page_content=doc.page_content, 
        metadata=ChunkMetadata(
            chunk_id=doc.metadata['chunk_id'],
            chunk_size=doc.metadata['chunk_size'],
            embedding_model=doc.metadata['embedding_model'],
            chunk_order=doc.metadata['chunk_order'],
            chunk_overlap=doc.metadata['chunk_overlap']
        )
    ) for doc in docs
]

In [132]:
for d in docs:
    print(d)

page_content='## Wat is een glucosesensor?' metadata={'chunk_size': 500, 'embedding_model': 'hkunlp/instructor-xl', 'chunk_order': 1544, 'chunk_overlap': 60, 'chunk_id': 1544}
page_content='## Wat is een circumcisie?' metadata={'chunk_size': 500, 'embedding_model': 'hkunlp/instructor-xl', 'chunk_order': 320, 'chunk_overlap': 60, 'chunk_id': 320}
page_content='## Wat is een facetinfiltratie?' metadata={'chunk_size': 500, 'embedding_model': 'hkunlp/instructor-xl', 'chunk_order': 1394, 'chunk_overlap': 60, 'chunk_id': 1394}
page_content='## Wat is een ERCP?' metadata={'chunk_size': 500, 'embedding_model': 'hkunlp/instructor-xl', 'chunk_order': 1326, 'chunk_overlap': 60, 'chunk_id': 1326}
page_content='## Wat is een fistel?' metadata={'chunk_size': 500, 'embedding_model': 'hkunlp/instructor-xl', 'chunk_order': 1414, 'chunk_overlap': 60, 'chunk_id': 1414}


In [127]:
def docs_to_str(l: List[langchain.schema.document.Document]) -> str:
    page_content: str = ""
    for i, d in enumerate(l): 
        # Page Content
        page_content += (
            (25 * "=") + 
            (f" Document {i+1} ") + 
            (25 * "=") + 
            '\n'
        )
        page_content += (
            # (25 * "=" ) + 
            (f"# Document {i+1} Metadata ") + 
            # (5 * "=" ) + 
            '\n'
        )
        # Metadata 
        for k, v in d.metadata.items():
            page_content += (f"- {k}: {v} \n")
        
        page_content += (
            # (25 * "=" ) + 
            (f"# Document {i+1}'s Content") + 
            # (5 * "=" ) + 
            '\n'
        )
        page_content += d.page_content
        page_content += '\n'
    return page_content


In [137]:
def chunk_paths_to_docs(chunk_paths: List[object]) -> List[Document]:
    """
        For now, this matches chunk_paths of type (Chunk)-rel-(WebPage)-()
    """
    # This returns paths, that we can turn into LangChain documents somehow. 
    path_docs: List[Document] = []

    # One result for every chunk (see above)
    for p in chunk_paths:
        chunk_path_str = '' 
        chunk_node = p['rel'][0] 
        chunk_text = chunk_node.get('text')
        
        # Build up metadata of Document object manually
        doc_meta = {
            'chunk_size': chunk_node.get('chunk_size'),
            'embedding_model': chunk_node.get('embedding_model'),
            'chunk_order': chunk_node.get('chunk_order'),
            'chunk_overlap': chunk_node.get('chunk_overlap'),
            'chunk_id': chunk_node.get('chunk_id'),
        }
        # Traverse path for metadata
        for i, o in enumerate(p['rel']): 
            # Create path representation
            if type(o) == dict:
                chunk_path_str += f'(Node)'
            elif type(o) == str:
                chunk_path_str += f'<-{o}-'
            
            # WebPage node data
            if i == 2:
                doc_meta['webpage_scrape_dt'] = o.get('scrape_dt')
                doc_meta['webpage_url'] = o.get('url')
                doc_meta['webpage_title'] = o.get('title')
            # Catalog node data
            elif i == 4: 
                doc_meta['catalog_url'] = o.get('url')
        
        # Add path structure as metadata
        doc_meta['path_context'] = chunk_path_str   
        
        # Extract metadata from traversal 
        path_docs.append(
            Document(
                page_content=chunk_text, 
                metadata=doc_meta
            )
        )
    return path_docs

In [ ]:
def get_neo4j_docs(question: str) -> List[langchain.schema.document.Document]:): 
    vec_chunk_paths = neo4j_graph.query(
        f"""
            CALL db.index.vector.queryNodes(
                "vi_chunk_embedding_cosine", 
                {RESULT_LIM},
                {embeddings.embed_query(VECTOR_SEARCH_QUERY)}
            ) 
            YIELD node, score
            WITH node, score
            ORDER BY score DESCENDING
            MATCH rel=(node:Chunk)<-[r*]-(:Catalog)
            RETURN rel
        """
    )

In [136]:
neo4j_retriever.get_relevant_documents(
   "Wat moet ik doen bij nierstenen?"
)

[Document(page_content='## Wat moet je doen?\n\nOm zoveel mogelijk informatie uit het onderzoek te krijgen, hebben we jouw medewerking nodig. Hierop moet je letten:', metadata={'chunk_size': 500, 'embedding_model': 'hkunlp/instructor-xl', 'chunk_order': 12, 'chunk_overlap': 60, 'chunk_id': 12}),
 Document(page_content='## Hoe bereid ik me het best voor?\n\n\nJe hoeft niet nuchter te zijn. Voor de start van het onderzoek bespreken we met jou de taken die je tijdens het onderzoek moet uitvoeren. Dit kunnen motorische taken zijn (waarbij je een bepaald lichaamsdeel moet bewegen) of cognitieve taken (waarbij je alleen moet denken) of beiden. Zorg ervoor dat je de instructies goed begrijpt en dat je weet wat er van jou verwacht wordt tijdens het onderzoek. Dit is nodig om het onderzoek te doen slagen.', metadata={'chunk_size': 500, 'embedding_model': 'hkunlp/instructor-xl', 'chunk_order': 1462, 'chunk_overlap': 60, 'chunk_id': 1462}),
 Document(page_content='* Bedek je neus en mond bij hoes

In [133]:
import textwrap
from langchain.chains import LLMChain

BASE_PROMPT_TEMPLATE_NL = """
JE BENT EEN ZIEKENHUISADMINISTRATEUR.
BEANTWOORD DE VRAAG DIE VOLGT ALLEEN OP BASIS VAN DE ONDERSTAANDE CONTEXT:
{context}

VRAAG: 
{question}

INSTRUCTIES BIJ HET ANTWOORDEN:
- ALGEMEEN:
    - GEEF DUIDELIJK AAN ALS HET ANTWOORD NIET GEVONDEN KAN WORDEN IN DE DATABASE
    - ANTWOORD ALTIJD IN HET {language}
- INDIEN JE NIETS VINDT IN DE CONTEXT, VERMELD DAN:
    - LIJST DE MEEST GELIJKE CONTEXT OP MET KOMMA'S
    - DE URL VAN DE CONTEXT
- INDIEN JE WEL IETS VINDT IN DE CONTEXT, VERMELD DAN:
    - LIJST DE RELEVANTE CONTEXT OP MET KOMMA'S
    - DE BRON (URL) VAN DE CONTEXT.
"""

# QUESTION =============================================
question = "Wat moet ik doen met"

## RETRIEVE DOCUMENTS =================
# Get docs from Neo4j
docs: List[langchain.schema.document.Document] = (
    neo4j_retriever.get_relevant_documents(
        question
    )
)

llm_prompt = PromptTemplate(
    input_variables=["context", "question", "language"],
    # partial_variables={"Customer_Name", "Customer_State", "Customer_Gender"},
    template=BASE_PROMPT_TEMPLATE_NL,
)

llm = ChatOpenAI(temperature=1)
llm_chain = LLMChain(
    prompt=llm_prompt, 
    llm=llm,
    verbose=True
)

llm_answer = llm_chain.invoke(
    input={
        "question": question,
        "context": docs_to_str(docs), 
        "language": "Nederlands"
    },
    return_only_outputs=True,
    include_run_info=False
)

# Show output
# wrapped_context = textwrap.wrap(llm_prompt.)

# Show answer 
wrapped_output = textwrap.wrap(llm_answer['text'], width=80)
for line in wrapped_output:
    print(line)



> Entering new LLMChain chain...
Prompt after formatting:

JE BENT EEN ZIEKENHUISADMINISTRATEUR.
BEANTWOORD DE VRAAG DIE VOLGT ALLEEN OP BASIS VAN DE ONDERSTAANDE CONTEXT:
========================= Document 1 =========================
# Document 1 Metadata 
- chunk_size: 500 
- embedding_model: hkunlp/instructor-xl 
- chunk_order: 1462 
- chunk_overlap: 60 
- chunk_id: 1462 
# Document 1's Content
## Hoe bereid ik me het best voor?


Je hoeft niet nuchter te zijn. Voor de start van het onderzoek bespreken we met jou de taken die je tijdens het onderzoek moet uitvoeren. Dit kunnen motorische taken zijn (waarbij je een bepaald lichaamsdeel moet bewegen) of cognitieve taken (waarbij je alleen moet denken) of beiden. Zorg ervoor dat je de instructies goed begrijpt en dat je weet wat er van jou verwacht wordt tijdens het onderzoek. Dit is nodig om het onderzoek te doen slagen.
========================= Document 2 =========================
# Document 2 Metadata 
- chunk_size: 500 
- embedd

In [76]:
print(docs_to_str(docs))

========================= Document 1 =========================
=============== Document 1 Metadata ===============
- chunk_size: 500 
- embedding_model: hkunlp/instructor-xl 
- chunk_order: 2431 
- chunk_overlap: 60 
- chunk_id: 2431 
=============== Document 1's Content===============
Er zijn verschillende types nierstenen (zes om precies te zijn), met elk hun eigen oorzaak en ontstaansmechanisme. Als je het type steen en de risicofactoren kent, kun je behandeling daarop afstemmen en is advies op maat mogelijk. 


Heel concreet geeft het metabool bilan een antwoord op de volgende vragen:


* Waarom maak ik nierstenen aan?
* Welke maatregelen zijn nodig om mijn kans op herval te verminderen.



## Voor wie?
========================= Document 2 =========================
=============== Document 2 Metadata ===============
- chunk_size: 500 
- embedding_model: hkunlp/instructor-xl 
- chunk_order: 3862 
- chunk_overlap: 60 
- chunk_id: 3862 
=============== Document 2's Content============